In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/llama3-finetuned-qs/transformers/default/1/adapter_model.safetensors
/kaggle/input/llama3-finetuned-qs/transformers/default/1/adapter_config.json
/kaggle/input/llama-3/transformers/8b-chat-hf/1/model.safetensors.index.json
/kaggle/input/llama-3/transformers/8b-chat-hf/1/model-00003-of-00004.safetensors
/kaggle/input/llama-3/transformers/8b-chat-hf/1/config.json
/kaggle/input/llama-3/transformers/8b-chat-hf/1/LICENSE
/kaggle/input/llama-3/transformers/8b-chat-hf/1/model-00001-of-00004.safetensors
/kaggle/input/llama-3/transformers/8b-chat-hf/1/model.py
/kaggle/input/llama-3/transformers/8b-chat-hf/1/USE_POLICY.md
/kaggle/input/llama-3/transformers/8b-chat-hf/1/tokenizer.json
/kaggle/input/llama-3/transformers/8b-chat-hf/1/tokenizer_config.json
/kaggle/input/llama-3/transformers/8b-chat-hf/1/example_text_completion.py
/kaggle/input/llama-3/transformers/8b-chat-hf/1/test_tokenizer.py
/kaggle/input/llama-3/transformers/8b-chat-hf/1/requirements.txt
/kaggle/input/llama-3/trans

In [2]:
%%capture
%pip install -U bitsandbytes
%pip install -U transformers
%pip install -U accelerate
%pip install -U peft
%pip install -U trl

In [3]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

hf_token = user_secrets.get_secret("HUGGINGFACE_TOKEN")
login(token = hf_token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [4]:
base_model = "/kaggle/input/llama-3/transformers/8b-chat-hf/1"
new_model = "/kaggle/input/llama3-finetuned-qs/transformers/default/1"

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from peft import PeftModel
import torch
from trl import setup_chat_format

2024-07-30 14:17:56.363856: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-30 14:17:56.363979: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-30 14:17:56.496086: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
# Reload tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(base_model)

base_model_reload = AutoModelForCausalLM.from_pretrained(
        base_model,
        return_dict=True,
        low_cpu_mem_usage=True,
        torch_dtype=torch.float16,
        device_map="auto",
        trust_remote_code=True,
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [7]:
# Merge adapter with base model
base_model_reload, tokenizer = setup_chat_format(base_model_reload, tokenizer)
model = PeftModel.from_pretrained(base_model_reload, new_model)

In [8]:
model = model.merge_and_unload()

In [10]:
# model.save_pretrained("llama-3-8b-chat-Qs-Ans-Quora")
# tokenizer.save_pretrained("llama-3-8b-Qs-Ans-Quora")

In [11]:
# model.push_to_hub("llama-3-8b-chat-Qs-Ans-Quora", use_temp_dir=False)
# tokenizer.push_to_hub("llama-3-8b-chat-Qs-Ans-Quora", use_temp_dir=False)

In [12]:
dataset_path = '/kaggle/input/quora-qs-ans/quora_dataset.csv'
df = pd.read_csv(dataset_path)
df=df[:10]

# Assuming your dataset has columns named 'qs' and 'ans'
questions = df['question'].tolist()
references = df['answer'].tolist()

generated_texts = []
for question in questions:
    messages = [{"role": "user", "content": question}]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    outputs = pipe(prompt, max_new_tokens=128, early_stopping=True, temperature=0.4, num_beams=3)
    generated_text = outputs[0]["generated_text"]
    generated_texts.append(generated_text)

df['generated'] = generated_texts


/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.4` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [13]:
# from rouge_score import rouge_scorer

# # Initialize the ROUGE scorer
# scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# # Calculate ROUGE scores for each pair of texts
# scores = [scorer.score(ref, gen) for ref, gen in zip(df['ans'], df['generated'])]

# # Convert the scores to a DataFrame for easier analysis
# scores_df = pd.DataFrame(scores)

# # Display the scores
# import ace_tools as tools; tools.display_dataframe_to_user(name="ROUGE Scores", dataframe=scores_df)
# print(scores_df)


In [14]:
pip install rouge_score

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=c55c2307e69bede65db3c0ea25f0f54c83459cd16fea86d656de57ab379d1177
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
Note: you may need to restart the kernel to use updated packages.


In [16]:
from rouge_score import rouge_scorer

# Function to compute ROUGE score
def compute_rouge(predictions, references):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = []
    for pred, ref in zip(predictions, references):
        score = scorer.score(ref, pred)
        scores.append(score)
    return scores

# Compute ROUGE scores
rouge_scores = compute_rouge(df['generated'], df['answer'])

# Initialize variables to store total f-measure scores
total_rouge1_fmeasure = 0
total_rouge2_fmeasure = 0
total_rougeL_fmeasure = 0

# Sum the f-measure scores
for score in rouge_scores:
    total_rouge1_fmeasure += score['rouge1'].fmeasure
    total_rouge2_fmeasure += score['rouge2'].fmeasure
    total_rougeL_fmeasure += score['rougeL'].fmeasure

# Calculate average f-measure scores
num_samples = len(rouge_scores)
average_rouge1_fmeasure = total_rouge1_fmeasure / num_samples
average_rouge2_fmeasure = total_rouge2_fmeasure / num_samples
average_rougeL_fmeasure = total_rougeL_fmeasure / num_samples

# Display the average f-measure scores
print("Average ROUGE Scores:")
print(f"ROUGE-1: {average_rouge1_fmeasure*100:.2f}")
print(f"ROUGE-2: {average_rouge2_fmeasure*100:.2f}")
print(f"ROUGE-L: {average_rougeL_fmeasure*100:.2f}")

Average ROUGE Scores:
ROUGE-1: 8.23
ROUGE-2: 1.60
ROUGE-L: 7.60
